In [ ]:
ALPHA_VANTAGE_API_KEY = "2TF07MIMWVNCRG7I"  # Replace with your API key

In [8]:
import yfinance as yf
import pandas as pd
import datetime
import time
from pandas_datareader import data as pdr
from alpha_vantage.foreignexchange import ForeignExchange

# Set Alpha Vantage API Key
ALPHA_VANTAGE_API_KEY = "2TF07MIMWVNCRG7I"  # Replace with your API key

df = pd.read_csv("fomc_meetings_corrected.csv")

def get_fed_meeting_dates(years=15):
    """Returns a list of Federal Reserve meeting dates for the past X years."""
    return df['meeting_end'].tolist()

def get_date_range(meeting_date):
    """Get 15 days before and after the meeting."""
    meeting_dt = datetime.datetime.strptime(meeting_date, "%Y-%m-%d")
    start_date = (meeting_dt - datetime.timedelta(days=15)).strftime("%Y-%m-%d")
    end_date = (meeting_dt + datetime.timedelta(days=15)).strftime("%Y-%m-%d")
    return start_date, end_date

def download_equity_data(symbols, start, end):
    """Download S&P 500, Nasdaq, Dow Jones data from Yahoo Finance."""
    data = {}
    for symbol in symbols:
        df = yf.download(symbol, start=start, end=end)

        print(f"Downloaded data for {symbol}:")
        print(df.head())  # Debug print

        # Handle MultiIndex issue
        if isinstance(df.columns, pd.MultiIndex):
            df = df.xs('Close', level=0, axis=1)  # Extract Close prices

        # Ensure 'Adj Close' or 'Close' column is used
        if 'Adj Close' in df.columns:
            data[symbol] = df[['Adj Close']]
        elif 'Close' in df.columns:
            data[symbol] = df[['Close']]
        else:
            print(f"Warning: Neither 'Adj Close' nor 'Close' found for {symbol}. Available columns: {df.columns}")
            data[symbol] = df  # Store full data for debugging

    return data

def download_bond_data(start, end):
    """Download U.S. Treasury bond yield data from FRED."""
    symbols = ["DGS10"]  # 10-year Treasury yield
    bond_data = {}
    for symbol in symbols:
        df = pdr.get_data_fred(symbol, start, end)
        bond_data[symbol] = df
    return bond_data

def download_forex_data(pairs, start, end):
    """Download forex data from Alpha Vantage."""
    fx = ForeignExchange(key=ALPHA_VANTAGE_API_KEY)
    forex_data = {}

    for pair in pairs:
        base, quote = pair.split('/')
        try:
            data, _ = fx.get_currency_exchange_daily(from_symbol=base, to_symbol=quote, outputsize='full')
            if not data:
                print(f"Warning: No data received for {pair}")
                continue

            df = pd.DataFrame.from_dict(data, orient='index')
            
            # Ensure column conversion
            df = df.rename(columns={"4. close": "Close"})
            df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
            
            # Convert index to datetime and sort it
            df.index = pd.to_datetime(df.index)
            df = df.sort_index()

            # Ensure date range selection works properly
            df = df.loc[start:end]

            forex_data[pair] = df[['Close']]
        except Exception as e:
            print(f"Error downloading {pair}: {e}")
        
        time.sleep(12)  # Avoid API rate limits
    return forex_data

def download_gold_data(start, end):
    """Download gold prices using Yahoo Finance."""
    gold = yf.download("GC=F", start=start, end=end)
    print("Downloaded Gold Data:")
    print(gold.head())  # Debug print
    if 'Adj Close' in gold.columns:
        return gold[['Adj Close']]
    else:
        print(f"Warning: 'Adj Close' not found in gold data. Available columns: {gold.columns}")
        return gold

def main():
    fed_meeting_dates = get_fed_meeting_dates()
    
    all_data = []
    for meeting_date in fed_meeting_dates:
        start_date, end_date = get_date_range(meeting_date)
        
        print(f"Downloading data for {meeting_date} ({start_date} to {end_date})")
        
        # Download data
        equities = download_equity_data(["^GSPC", "^IXIC", "^DJI"], start_date, end_date)
        bonds = download_bond_data(start_date, end_date)
        forex = download_forex_data(["USD/EUR", "USD/JPY"], start_date, end_date)
        gold = download_gold_data(start_date, end_date)
        
        # Combine data
        for symbol, df in equities.items():
            df['Type'] = symbol
            df['Date'] = df.index
            all_data.append(df.reset_index(drop=True))
        
        for symbol, df in bonds.items():
            df['Type'] = symbol
            df['Date'] = df.index
            all_data.append(df.reset_index(drop=True))
        
        for pair, df in forex.items():
            df['Type'] = pair
            df['Date'] = df.index
            all_data.append(df.reset_index(drop=True))
        
        if not gold.empty:
            gold['Type'] = "Gold"
            gold['Date'] = gold.index
            all_data.append(gold.reset_index(drop=True))
        
        time.sleep(10)  # Avoid API rate limits
    
    # Merge all data
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_csv("market_data.csv", index=False)
        print("Data collection complete. Saved to market_data.csv")
    else:
        print("No data collected.")

if __name__ == "__main__":
    main()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Downloaded data for ^GSPC:
Price             Close         High          Low         Open      Volume
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC
Date                                                                      
2010-01-12  1136.219971  1143.810059  1131.770020  1143.810059  4716160000
2010-01-13  1145.680054  1148.400024  1133.180054  1137.310059  4170360000
2010-01-14  1148.459961  1150.410034  1143.800049  1145.680054  3915200000
2010-01-15  1136.030029  1147.770020  1131.390015  1147.719971  4758730000
2010-01-19  1150.229980  1150.449951  1135.770020  1136.030029  4724830000
            (  'High', '^GSPC'),
            (   'Low', '^GSPC'),
            (  'Open', '^GSPC'),
            ('Volume', '^GSPC')],
           names=['Price', 'Ticker'])
Downloaded data for ^IXIC:
Price             Close         High          Low         Open      Volume
Ticker            ^IXIC        ^IXIC        ^IXIC        ^IXIC       ^IXIC
Date                     


[*********************100%***********************]  1 of 1 completed


Downloaded Gold Data:
Price             Close         High          Low         Open Volume
Ticker             GC=F         GC=F         GC=F         GC=F   GC=F
Date                                                                 
2010-01-12  1128.900024  1157.199951  1127.199951  1128.900024     51
2010-01-13  1136.400024  1136.400024  1121.000000  1136.400024     58
2010-01-14  1142.599976  1145.900024  1132.800049  1137.000000     81
2010-01-15  1130.099976  1133.400024  1127.199951  1132.800049     50
2010-01-19  1139.699951  1139.699951  1130.500000  1139.699951     22
            (  'High', 'GC=F'),
            (   'Low', 'GC=F'),
            (  'Open', 'GC=F'),
            ('Volume', 'GC=F')],
           names=['Price', 'Ticker'])


...

Downloaded data for ^GSPC:
Price             Close         High          Low         Open      Volume
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC
Date                                                                      
2024-12-03  6049.879883  6052.069824  6033.390137  6042.970215  4095000000
2024-12-04  6086.490234  6089.839844  6061.060059  6069.390137  4003390000
2024-12-05  6075.109863  6094.549805  6072.899902  6089.029785  4212020000
2024-12-06  6090.270020  6099.970215  6079.979980  6081.379883  3924830000
2024-12-09  6052.850098  6088.509766  6048.629883  6083.009766  4556460000
            (  'High', '^GSPC'),
            (   'Low', '^GSPC'),
            (  'Open', '^GSPC'),
            ('Volume', '^GSPC')],
           names=['Price', 'Ticker'])



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded data for ^IXIC:
Price              Close          High           Low          Open      Volume
Ticker             ^IXIC         ^IXIC         ^IXIC         ^IXIC       ^IXIC
Date                                                                          
2024-12-03  19480.910156  19486.150391  19340.410156  19364.339844  6005160000
2024-12-04  19735.119141  19741.759766  19575.400391  19587.480469  7870030000
2024-12-05  19700.259766  19790.029297  19688.589844  19756.400391  7229510000
2024-12-06  19859.769531  19863.150391  19734.419922  19743.960938  6787610000
2024-12-09  19736.689453  19872.789062  19698.050781  19824.089844  7884620000
            (  'High', '^IXIC'),
            (   'Low', '^IXIC'),
            (  'Open', '^IXIC'),
            ('Volume', '^IXIC')],
           names=['Price', 'Ticker'])
Downloaded data for ^DJI:
Price              Close          High           Low          Open     Volume
Ticker              ^DJI          ^DJI          ^DJI          ^DJI

[*********************100%***********************]  1 of 1 completed


Downloaded Gold Data:
Price             Close         High          Low         Open Volume
Ticker             GC=F         GC=F         GC=F         GC=F   GC=F
Date                                                                 
2024-12-03  2644.699951  2654.699951  2636.000000  2639.000000   1062
2024-12-04  2653.800049  2658.300049  2637.000000  2639.199951    249
2024-12-05  2626.600098  2655.000000  2625.699951  2652.399902    573
2024-12-06  2638.600098  2643.100098  2617.500000  2620.199951    571
2024-12-09  2664.899902  2677.100098  2630.800049  2632.100098    935
            (  'High', 'GC=F'),
            (   'Low', 'GC=F'),
            (  'Open', 'GC=F'),
            ('Volume', 'GC=F')],
           names=['Price', 'Ticker'])
Data collection complete. Saved to market_data.csv


## Rename to be more readable

In [1]:
# convert the result to more readable format
import pandas as pd

# Load the existing CSV
df = pd.read_csv('market_data.csv')

# Mapping for symbols to readable names
symbol_mapping = {
    "^GSPC": "SP500",
    "^IXIC": "Nasdaq",
    "^DJI": "DowJones",
    "GC=F": "Gold",
    "DGS10": "10Y_Treasury_Yield",
    "USD/EUR": "USD_EUR",
    "USD/JPY": "USD_JPY"
}

def rename_columns(columns):
    new_columns = []
    for col in columns:
        # Remove parentheses and split if needed
        if col.startswith("(") and col.endswith(")"):
            parts = col.strip("()").replace("'", "").split(", ")
            if len(parts) == 2:
                col_name, symbol = parts
                readable_symbol = symbol_mapping.get(symbol, symbol)
                new_columns.append(f"{readable_symbol}_{col_name}")
            else:
                new_columns.append(col)
        else:
            # For non-tuple columns like 'DGS10', 'Type', 'Date'
            readable_col = symbol_mapping.get(col, col)
            new_columns.append(readable_col)
    return new_columns

# Apply the renaming
df.columns = rename_columns(df.columns)



Columns renamed and saved to market_data_renamed.csv


## Clean Type and Date, Consolidate

In [4]:
# Drop the Type columns as they are redundant
df.drop(columns=['_Type', 'Type'], inplace=True)

# Fill missing Date values to align all rows
df['Date'] = df['Date'].combine_first(df['_Date'])
df.drop(columns=['_Date'], inplace=True)

# Convert Date to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Group by Date to ensure one row per date, taking non-null values from each group
consolidated_df = df.groupby('Date').first().reset_index()

# Set Date as the index
consolidated_df.set_index('Date', inplace=True)

# Save the cleaned dataset
consolidated_df.to_csv('market_data_consolidated.csv')


In [7]:
# check if any col in consolidated_df has missing values
missing_values = consolidated_df.isnull().sum()
print(missing_values)

SP500_Close            171
SP500_High             171
SP500_Low              171
SP500_Open             171
SP500_Volume           171
Nasdaq_Close           171
Nasdaq_High            171
Nasdaq_Low             171
Nasdaq_Open            171
Nasdaq_Volume          171
DowJones_Close         171
DowJones_High          171
DowJones_Low           171
DowJones_Open          171
DowJones_Volume        171
10Y_Treasury_Yield      66
Close                 2760
Gold_Close             173
Gold_High              173
Gold_Low               173
Gold_Open              173
Gold_Volume            173
dtype: int64


## Remove Incorrect Close Column

In [9]:
# delete col called Close
consolidated_df.drop(columns=['Close'], inplace=True)
# drop any rows that have missing values
consolidated_df.dropna(inplace=True)
# save to new
consolidated_df.to_csv('market_data_cleaned.csv')